# Задание 2: Полигоны влияния

## Условие
Яндекс.Еда осуществляет доставку еды из ресторанов. При этом у каждого ресторана есть зона, в рамках которой осуществляется доставка. Зона представляет собой полигон (заданы координаты его вершин). Пользователь в зависимости от своего местоположения (координат точки) видит разное количество доступных ресторанов. Нам важно, чтобы у каждого пользователя было достаточное количество ресторанов для выбора. Задача заключается в том, чтобы для каждого пользователя посчитать доступное ему количество ресторанов.

In [38]:
import numpy as np
import pandas as pd
import math

### Имеющиеся данные
Данные, которые есть (для простоты в формате .csv, несколько первых строк):

* *user_coordinates.csv* (примерно 300 тыс. строк, **user_id** – идентификатор пользователя)

In [39]:
users = pd.read_csv("user_coordinates.csv")
users['user_pos'] = list(zip(round(users.loc_lat, 6), round(users.loc_lon, 6)))
users.head()

,user_id,loc_lat,loc_lon,user_pos
0,1,55.737564,37.345186,"(55.737564, 37.345186)"
1,2,56.234564,37.234590,"(56.234564, 37.23459)"
2,3,55.234578,36.295745,"(55.234578, 36.295745)"
3,4,2.000000,2.000000,"(2.0, 2.0)"


* *place_zone_coordinates.csv* (примерно 500 тыс. строк, **place_id** – идентификатор ресторана, **point_number** – порядковый номер вершины полигона)

In [16]:
places = pd.read_csv("place_zone_coordinates.csv").sort_values(['place_id', 'point_number'])
places['pos'] = list(zip(round(places.loc_lat, 6), round(places.loc_lon, 6)))
places

,place_id,loc_lat,loc_lon,point_number,pos
0,1,55.747022,37.787073,0,"(55.747022, 37.787073)"
1,1,55.751713,37.784328,1,"(55.751713, 37.784328)"
2,1,55.753878,37.777638,2,"(55.753878, 37.777638)"
3,1,55.751031,37.779351,3,"(55.751031, 37.779351)"
4,2,55.803885,37.458311,0,"(55.803885, 37.458311)"
5,2,55.808677,37.464054,1,"(55.808677, 37.464054)"
6,2,55.809763,37.461314,2,"(55.809763, 37.461314)"
7,2,55.810840,37.458654,3,"(55.81084, 37.458654)"
8,3,1.000000,1.000000,0,"(1.0, 1.0)"
9,3,1.000000,3.000000,1,"(1.0, 3.0)"


### Формат ответа
id,number_of_places_available\
1,2\
2,19\
3,0

## Решение
Нам понадобится функция, которая проверяет, принадлежит ли точка данному полигону. Также из-за большого количества полигонов и точек, суммарное количество взаимодействий будет большим, поэтому нужно будет снизить количество вычислений  с помощью интуитивно понятной предпроверки.

In [82]:
poly_lists = places.groupby('place_id').apply(lambda x: list(zip(round(x.loc_lat, 6), round(x.loc_lon, 6))))
mins = places.groupby('place_id').min()
maxes = places.groupby('place_id').max()
polys = pd.DataFrame({'poly': poly_lists,
                      'bounds': list(zip(list(zip(mins.loc_lat, mins.loc_lon)),
                                         list(zip(maxes.loc_lat, maxes.loc_lon))))})
polys.head()

,poly,bounds
place_id,,
1,"[(55.747022, 37.787073), (55.751713, 37.784328...","((55.747021999999994, 37.777638), (55.753878, ..."
2,"[(55.803885, 37.458311), (55.808677, 37.464054...","((55.803885, 37.458311), (55.81084, 37.464054))"
3,"[(1.0, 1.0), (1.0, 3.0), (3.0, 3.0), (3.0, 2.0...","((1.0, 1.0), (5.0, 3.0))"


### Быстрая проверка принадлежности
У нас большое количество полигонов и точек, поэтому нужно искать пути ускорить расчёты. Простая проверка на то, входит ли точка в прямоугольник, содержащий рассматриваемый полигон, позволяет отсеять огромное число полигонов без существенных вычислений.

In [83]:
def fast_check(point, bounds):
    return point[0] < bounds[0][0] or point[0] > bounds[1][0] or \
           point[1] < bounds[0][1] or point[1] > bounds[1][1]

### Проверка принадлежности 
Для проверки принадлежности точки произвольному полигону используется метод трассировки луча. Если горизонтальный луч, исходящий из данной точки, пересекается с ребрами полигона нечётное число раз, то точка лежит внутри полигона.

In [79]:
def true_sign(s):
    '''Стандартный signum'''
    if s > 0: return 1
    if s < 0: return -1
    return 0

def poly_edges(poly):
    '''Генератор рёбер для данного полигона'''
    poly_len = len(poly)
    for start_pid in range(poly_len):
        yield (poly[start_pid], poly[(start_pid+1)%poly_len])

def edge_type(point, edge):
    '''Оптимизированная функция проверки отношения точка-ребро.
       Спасибо товарищу @halyavin: habr.com/ru/post/161237.
    '''
    e_org, e_dest = edge
    
    org_x, org_y = e_org[0]-point[0], e_org[1]-point[1]
    dest_x, dest_y = e_dest[0]-point[0], e_dest[1]-point[1]
    
    if org_y * dest_y > 0: return 1
    
    sign = true_sign(org_x * dest_y - org_y * dest_x)
    if sign == 0:
        if org_x * dest_x <= 0:
            return 0
        return 1
    
    if org_y < 0:
        return -sign
    if dest_y < 0:
        return sign
    
    return 1

def point_in_poly(point, poly):
    '''Проверка на принадлежность точки полигону'''
    point_stat = 1
    for edge in poly_edges(poly):
        point_stat *= edge_type(point, edge)
        if not point_stat:
            return point_stat
    return point_stat

In [80]:
def count_polys(point):
    '''Apply-функция для подсчёта полигонов для точки'''
    total = 0
    for poly_idx in polys.index:
        if fast_check(point, polys.loc[poly_idx].bounds):
            continue
        total += point_in_poly(point, polys.loc[poly_idx].poly) < 1
    return total

users['number_of_places_available'] = users.user_pos.apply(count_polys)

In [84]:
answer = users[['user_id', 'number_of_places_available']].rename(columns={'user_id':'id'}).set_index('id')
answer.to_csv('answer.csv')
answer.head()

,number_of_places_available
id,
1,0
2,0
3,0
4,1
